In [61]:
# 导入必要的库
import pandas as pd
import numpy as np

# 读取CSV文件
df = pd.read_csv('charging_station_data.csv')
df = df.reset_index(drop=True)
# 显示数据的基本信息
print(df.info())

# 检查tariff_amount列的空值数量
print("\n空值数量:")
print(df['tariff_amount'].isnull().sum())

# 定义一个函数来填充tariff_amount的空值
def fill_tariff_amount(group):
    mask = group['tariff_amount'].isnull()
    group.loc[mask, 'tariff_amount'] = group['tariff_amount'].mean()
    return group

# 按max_chargerate, plug_type, connector_type分组，并应用填充函数
df = df.groupby(['max_chargerate', 'plug_type', 'connector_type'], group_keys=False).apply(fill_tariff_amount)

# 再次检查tariff_amount列的空值数量
print("\n填充后的空值数量:")
print(df['tariff_amount'].isnull().sum())

# 显示填充后的数据样本
print("\n填充后的数据样本:")
print(df[['max_chargerate', 'plug_type', 'connector_type', 'tariff_amount']].sample(10))

# 检查填充是否正确
print("\n每组的平均tariff_amount:")
print(df.groupby(['max_chargerate', 'plug_type', 'connector_type'])['tariff_amount'].mean())

/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_20621/2155779481.py:6: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('charging_station_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3011648 entries, 0 to 3011647
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   city_id               int64  
 1   station_name          object 
 2   connector_id          int64  
 3   coordinates_x         float64
 4   coordinates_y         float64
 5   postcode              object 
 6   tariff_amount         float64
 7   tariff_connectionfee  float64
 8   max_chargerate        object 
 9   plug_type             object 
 10  connector_type        object 
 11  datetime              object 
 12  weather               object 
 13  is_available          int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 321.7+ MB
None

空值数量:
97707


/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_20621/2155779481.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['max_chargerate', 'plug_type', 'connector_type'], group_keys=False).apply(fill_tariff_amount)



填充后的空值数量:
0

填充后的数据样本:
        max_chargerate    plug_type connector_type  tariff_amount
2339649             22  type_2_plug             AC           0.50
514134              50          ccs          Rapid           0.55
1709824              7  type_2_plug             AC           0.40
324294              22  type_2_plug             AC           0.45
712716              22  type_2_plug             AC           0.45
395371              22  type_2_plug             AC           0.50
1838219             50      chademo          Rapid           0.70
1160247             22  type_2_plug             AC           0.31
119324              50      chademo          Rapid           0.70
1920198              7  type_2_plug             AC           0.45

每组的平均tariff_amount:
max_chargerate  plug_type    connector_type        
0               type_2_plug  AC                        0.210000
7               UNKNOWN      Rapid                     0.500000
                type_2_plug  AC                  

In [62]:
# 检查 tariff_connectionfee 列的空值数量
print("\ntariff_connectionfee 列的空值数量(填充前):")
print(df['tariff_connectionfee'].isnull().sum())

# 将 tariff_connectionfee 列的空值填充为 0
df['tariff_connectionfee'] = df['tariff_connectionfee'].fillna(0)

# 再次检查 tariff_connectionfee 列的空值数量,确保填充成功
print("\ntariff_connectionfee 列的空值数量(填充后):")
print(df['tariff_connectionfee'].isnull().sum())

# 显示填充后的数据样本
print("\n填充后的数据样本:")
print(df[['station_name', 'tariff_amount', 'tariff_connectionfee']].sample(10))

# 检查 tariff_connectionfee 列的基本统计信息
print("\ntariff_connectionfee 列的基本统计信息:")
print(df['tariff_connectionfee'].describe())


tariff_connectionfee 列的空值数量(填充前):
1902719

tariff_connectionfee 列的空值数量(填充后):
0

填充后的数据样本:
        station_name  tariff_amount  tariff_connectionfee
916221         52025       0.370000                  0.00
785544         51750       0.470000                  0.38
2544932        61219       0.480000                  0.00
1785606        53893       0.500000                  0.50
2379781        60920       0.400000                  1.00
38294          50289       0.419101                  0.00
2778514        61843       0.480000                  0.00
2221761        60573       0.450000                  0.00
1692191        53583       0.400000                  0.00
1682355        53557       0.600000                  0.00

tariff_connectionfee 列的基本统计信息:
count    3.011648e+06
mean     1.247873e-01
std      3.472551e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.000000e+00
Name: tariff_connectionfee, dtype: float64


In [63]:
# 显示原始数据的行数
print(f"原始数据行数: {len(df)}")

# 检查每列中 'UNKNOWN' 的数量
columns_to_check = ['max_chargerate', 'plug_type', 'connector_type']
for col in columns_to_check:
    unknown_count = (df[col] == 'UNKNOWN').sum()
    print(f"{col} 中 'UNKNOWN' 的数量: {unknown_count}")

# 删除包含 'UNKNOWN' 的行
df_cleaned = df[~df[columns_to_check].isin(['UNKNOWN']).any(axis=1)]

# 显示清理后的数据行数
print(f"\n清理后的数据行数: {len(df_cleaned)}")

# 再次检查是否还有 'UNKNOWN' 值
for col in columns_to_check:
    unknown_count = (df_cleaned[col] == 'UNKNOWN').sum()
    print(f"{col} 中剩余 'UNKNOWN' 的数量: {unknown_count}")

# 显示清理后的数据样本
print("\n清理后的数据样本:")
print(df_cleaned[columns_to_check + ['tariff_amount']].sample(10))

# 将 max_chargerate 转换为数值类型（如果还不是的话）
df_cleaned['max_chargerate'] = pd.to_numeric(df_cleaned['max_chargerate'], errors='coerce')

# 显示清理后数据的基本统计信息
print("\n清理后数据的基本统计信息:")
print(df_cleaned[columns_to_check + ['tariff_amount']].describe())

# 更新原始 DataFrame
df = df_cleaned.copy()

原始数据行数: 3011648
max_chargerate 中 'UNKNOWN' 的数量: 979
plug_type 中 'UNKNOWN' 的数量: 19327
connector_type 中 'UNKNOWN' 的数量: 978

清理后的数据行数: 2991343
max_chargerate 中剩余 'UNKNOWN' 的数量: 0
plug_type 中剩余 'UNKNOWN' 的数量: 0
connector_type 中剩余 'UNKNOWN' 的数量: 0

清理后的数据样本:
        max_chargerate    plug_type connector_type  tariff_amount
449882              22  type_2_plug             AC           0.49
598809              50          ccs          Rapid           0.70
1322893              7  type_2_plug             AC           0.47
676878              51          ccs          Rapid           0.70
1579160             22  type_2_plug             AC           0.42
64218               22  type_2_plug             AC           0.45
697264              22  type_2_plug          Rapid           0.45
1282177             22  type_2_plug             AC           0.48
1313404              7  type_2_plug             AC           0.40
2312293             22  type_2_plug             AC           0.47


/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_20621/2060580324.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['max_chargerate'] = pd.to_numeric(df_cleaned['max_chargerate'], errors='coerce')



清理后数据的基本统计信息:
       max_chargerate  tariff_amount
count    2.991343e+06   2.991343e+06
mean     2.324927e+01   4.701428e-01
std      1.585277e+01   4.498947e-01
min      0.000000e+00   0.000000e+00
25%      7.000000e+00   4.000000e-01
50%      2.200000e+01   4.500000e-01
75%      2.200000e+01   5.500000e-01
max      1.500000e+02   3.400000e+01


In [65]:
# 确保 datetime 列是日期时间格式
df['datetime'] = pd.to_datetime(df['datetime'])

# 按 station_name 和 datetime 排序
df = df.sort_values(['station_name', 'datetime'])

# 检查 weather 列的空值数量（处理前）
print("Weather 列空值数量（处理前）:")
print(df['weather'].isnull().sum())

# 定义一个函数来填充 weather 列的空值
def fill_weather(group):
    group['weather'] = group['weather'].fillna(method='ffill')
    group['weather'] = group['weather'].fillna(method='bfill')
    return group

# 对每个充电站分组应用填充函数
df = df.groupby('station_name').apply(fill_weather)

# 重置索引
df = df.reset_index(drop=True)

# 检查 weather 列的空值数量（处理后）
print("\nWeather 列空值数量（处理后）:")
print(df['weather'].isnull().sum())

# 打印索引信息以确认
print("\n索引信息:")
print(df.index)

Weather 列空值数量（处理前）:
493831


/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_20621/1847465488.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['weather'] = group['weather'].fillna(method='ffill')
/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_20621/1847465488.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['weather'] = group['weather'].fillna(method='bfill')
/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_20621/1847465488.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('stati


Weather 列空值数量（处理后）:
0

索引信息:
RangeIndex(start=0, stop=2991343, step=1)


In [66]:
# 定义一个函数来去除字符串两端的引号
def remove_quotes(text):
    if isinstance(text, str):
        return text.strip("'\"")
    return text

# 应用函数到 weather 列
df['weather'] = df['weather'].apply(remove_quotes)

# 检查处理结果
print("处理后的 weather 列唯一值：")
print(df['weather'].unique())

# 显示天气种类及其频率
print("\n天气种类及其频率：")
print(df['weather'].value_counts())

# 显示包含引号的天气记录（如果还有的话）
quotes = df[df['weather'].str.contains("'|\"", na=False)]
if not quotes.empty:
    print("\n仍然包含引号的记录：")
    print(quotes[['station_name', 'datetime', 'weather']])
else:
    print("\n所有引号已成功移除。")

# 显示处理后的数据样本
print("\n处理后的数据样本：")
print(df[['station_name', 'datetime', 'weather']].sample(10))

处理后的 weather 列唯一值：
['Rain' 'Clouds' 'Fog' 'Clear' 'Drizzle' 'Mist' 'Haze']

天气种类及其频率：
weather
Clouds     2454997
Rain        330784
Clear       180369
Mist         12628
Drizzle       6734
Haze          4554
Fog           1277
Name: count, dtype: int64

所有引号已成功移除。

处理后的数据样本：
        station_name            datetime weather
2470862        61135 2024-06-23 17:00:00  Clouds
459013         51214 2024-07-01 12:00:00  Clouds
802253         51854 2024-06-22 20:00:00  Clouds
935487         52072 2024-06-16 11:00:00    Rain
77674          50407 2024-06-18 04:00:00  Clouds
1811520        54012 2024-06-25 23:00:00  Clouds
50160          50327 2024-07-01 16:00:00  Clouds
98614          50462 2024-06-26 01:00:00  Clouds
1016355        52223 2024-07-01 20:00:00  Clouds
2596568        61418 2024-06-23 19:00:00   Clear


In [67]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_holiday'] = ((df['datetime'].dt.dayofweek.isin([5, 6])) | 
                    ((df['datetime'].dt.dayofweek == 4) & (df['hour'] >= 18))).astype(int)
df['time_of_day'] = pd.cut(df['hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])
df['is_work_hour'] = ((df['day_of_week'].isin([0,1,2,3,4])) & (df['hour'].between(9, 17))).astype(int)

In [68]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns


In [69]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# 创建 OneHotEncoder 对象
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# 对 'plug_type' 进行独热编码
plug_type_encoded = ohe.fit_transform(df[['plug_type']])
plug_type_columns = [f'plug_type_{cat}' for cat in ohe.categories_[0]]
df_plug_type = pd.DataFrame(plug_type_encoded, columns=plug_type_columns, index=df.index)

# 对 'connector_type' 进行独热编码
connector_type_encoded = ohe.fit_transform(df[['connector_type']])
connector_type_columns = [f'connector_type_{cat}' for cat in ohe.categories_[0]]
df_connector_type = pd.DataFrame(connector_type_encoded, columns=connector_type_columns, index=df.index)

# 对 'weather' 进行独热编码
weather_encoded = ohe.fit_transform(df[['weather']])
weather_columns = [f'weather_{cat}' for cat in ohe.categories_[0]]
df_weather = pd.DataFrame(weather_encoded, columns=weather_columns, index=df.index)

# 将编码后的特征合并到原始数据框
df = pd.concat([df, df_plug_type, df_connector_type, df_weather], axis=1)

In [70]:
# 确保 station_name 和 connector_id 都是字符串类型
df['station_name'] = df['station_name'].astype(str)
df['connector_id'] = df['connector_id'].astype(str)

# 使用 str.cat 方法创建 connector_unique_id
df['connector_unique_id'] = df['station_name'].str.cat(df['connector_id'], sep='_')

In [71]:
df['connector_avg_usage'] = 1 - df.groupby('connector_unique_id')['is_available'].transform('mean')

In [76]:
df['station_avg_usage'] = 1 - df.groupby('station_name')['is_available'].transform('mean')

In [79]:
df = df.sort_values(['station_name', 'connector_id', 'datetime'])

In [83]:
df['usage_last_24h'] = df.groupby('connector_unique_id')['is_available'].transform(
    lambda x: 1 - x.rolling(window=24, min_periods=1).mean()
)

In [84]:
print(df.index.names)  # 查看索引名称
print(df.columns)      # 查看列名
print(df.head())       # 查看前几行数据

[None]
Index(['city_id', 'station_name', 'connector_id', 'coordinates_x',
       'coordinates_y', 'postcode', 'tariff_amount', 'tariff_connectionfee',
       'max_chargerate', 'plug_type', 'connector_type', 'datetime', 'weather',
       'is_available', 'hour', 'day_of_week', 'is_weekend', 'is_holiday',
       'time_of_day', 'is_work_hour', 'plug_type_ccs', 'plug_type_chademo',
       'plug_type_type_2_plug', 'connector_type_AC',
       'connector_type_AC Controller/Receiver', 'connector_type_Rapid',
       'connector_type_Ultra-Rapid', 'connector_type_iCharging',
       'weather_Clear', 'weather_Clouds', 'weather_Drizzle', 'weather_Fog',
       'weather_Haze', 'weather_Mist', 'weather_Rain', 'connector_unique_id',
       'connector_avg_usage', 'station_avg_usage', 'usage_last_24h'],
      dtype='object')
   city_id station_name connector_id  coordinates_x  coordinates_y  postcode  \
0       80        50035            1       -4.61764       57.21494  IV63 7YA   
2       80        50035 

In [85]:
# 显示处理后的前几行数据
print(df.head())

# 显示DataFrame的基本信息
print(df.info())


   city_id station_name connector_id  coordinates_x  coordinates_y  postcode  \
0       80        50035            1       -4.61764       57.21494  IV63 7YA   
2       80        50035            1       -4.61764       57.21494  IV63 7YA   
4       80        50035            1       -4.61764       57.21494  IV63 7YA   
6       80        50035            1       -4.61764       57.21494  IV63 7YA   
8       80        50035            1       -4.61764       57.21494  IV63 7YA   

   tariff_amount  tariff_connectionfee  max_chargerate    plug_type  ...  \
0           0.75                   1.5               7  type_2_plug  ...   
2           0.75                   1.5               7  type_2_plug  ...   
4           0.75                   1.5               7  type_2_plug  ...   
6           0.75                   1.5               7  type_2_plug  ...   
8           0.75                   1.5               7  type_2_plug  ...   

  weather_Clouds weather_Drizzle weather_Fog  weather_Haze  we

In [49]:
import os
from datetime import datetime

# 获取当前日期和时间
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

# 创建输出文件名
output_filename = f"cleaned_charging_station_data_{current_time}.csv"

# 确保输出目录存在
output_dir = "cleaned_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 构建完整的输出路径
output_path = os.path.join(output_dir, output_filename)

# 将数据框输出到CSV文件
df.to_csv(output_path, index=False)

print(f"数据已成功输出到文件：{output_path}")

数据已成功输出到文件：cleaned_data\cleaned_charging_station_data_20240710_193353.csv
